<a href="https://colab.research.google.com/github/yukinaga/gnn/blob/main/section_3/02_simple_gnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# シンプルなGNNの実装
PyTorch Geometricを使い、シンプルなGNNを実装します。  
学習にGPUを利用するので、「編集」→「ノートブックの設定」の「ハードウェアアクセラレーター」で「GPU」を選択しましょう。

## Google ドライブとの連携  
（2022.12.10更新 バージョンアップによりPyTorch Geometricのインストール速度が向上したため、Google Driveを使う必要がなくなりました）  
~~今回はライブラリのサイズが大きく毎回インストールするのが大変なので、Googleドライブに保存します。  
まずは以下のコードを実行し、認証コードを使用してGoogle ドライブをマウントします。~~

In [ ]:
# from google.colab import drive
# drive.mount("/content/drive/")

~~Googleドライブ上のパスを指定します。~~

In [ ]:
# dir_name = "Live/gnn_live"  # 好きなパスを設定してください
# package_path = "/content/drive/MyDrive/" + dir_name + "/packages/"

## PyTorch Geometricのインストール
GNN用のライブラリ「PyTorch Geometric」、および関連ライブラリをインストールします。  
~~既にGoogle Driveにこれらのライブラリがインストール済みであれば、以下のセルのコードを実行する必要はありません。~~

In [ ]:
!pip install pyg-lib torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.13.0+cu116.html
!pip install torch-geometric
!pip install scipy==1.8.0

~~Google Driveに保存したパッケージをシステムに追加します。  ~~

In [ ]:
# import sys

# sys.path.append(package_path)  

## データセットの読み込み
データセット、「Cora」を読み込みます。  
Coraデータセットは、2708の科学論文の引用ネットワークで構成されています。  
グラフで表した場合、ノードは各論文を、エッジは引用関係を表します。    
各論文は7つのクラスに分類されます。  
データセット内の各論文は、特徴量として単語の非存在or存在を示す0or1からなる単語ベクトルを持っています。  
ユニークな単語の数は1433です。  

以下のコードにより、Coraのデータセットを読み込みます。

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root="/tmp/Cora", name="Cora")
data = dataset[0]

グラフの情報を表示するための関数を設定します。

In [ ]:
def graph_info(data):

    print("ノードの数:", data.num_nodes)
    print("エッジの数:", data.num_edges)
    print("特徴量の数:", data.num_node_features)
    print("無向グラフか？:", data.is_undirected())
    print("孤立したノードが有るか？:", data.has_isolated_nodes())
    print("自己ループがあるか？:", data.has_self_loops())

    print()

    print("キー: ", data.keys)
    print("各ノードの特徴量")
    print(data["x"])
    print("各ノードのラベル")
    print(data["y"])
    print("各エッジ")
    print(data["edge_index"])

関数を使って、データセットの情報を表示します。  

In [ ]:
graph_info(data)

NetworkXとmatplotlibを使ってグラフを可視化します。

In [ ]:
from torch_geometric.utils import to_networkx
import networkx as nx
import matplotlib.pyplot as plt

data_nx = to_networkx(data)  # networkxのグラフに変換

plt.figure(figsize=(12, 10))
nx.draw(data_nx,
        node_color = data.y,
        node_size=10)
plt.show()

## モデルの構築
シンプルなGNNのモデルを構築します。  
今回は2層からなる「Graph Convolutional Networks」を実装します。  
層の実装には、`GCNConv()`を利用します。  
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GCNConv

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GCN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 32)
        self.relu = nn.ReLU()  # ReLU
        self.conv2 = GCNConv(32, dataset.num_classes)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index

        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)

        return x

net = GCN()
net.cuda()  #GPU対応

## 学習
訓練データを使い、モデルを訓練します。

In [ ]:
from torch import optim

data = data.cuda()  # GPU対応

# 交差エントロピー誤差関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters())

net.train()  # 訓練モード
for epoch in range(200):
    optimizer.zero_grad()  # ①勾配の初期化
    out = net(data)  # ②順伝播により予測値を得る
    loss = loss_fnc(out[data.train_mask], data.y[data.train_mask])  # ③予測値と正解値から誤差を計算

    loss.backward()  # ④誤差からバックプロパゲーションにより勾配を計算
    optimizer.step()  # ⑤最適化アルゴリズムによりパラメータを更新

## モデルの評価
テスト用のノードを使って精度を測定し、モデルを評価します。

In [ ]:
net.eval()  # 評価モード
pred = net(data).argmax(dim=1)
correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
accuracy = int(correct) / int(data.test_mask.sum())
print("正解率:", str(accuracy*100) + "%")